<a href="https://colab.research.google.com/github/DanRepositories/TareaIO-GA/blob/main/TareaIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carga del archivo de la instancia

In [8]:
from google.colab import files
instance_file_name = 'QAP_sko56_04_n'
upload = files.upload()

Saving QAP_sko56_04_n to QAP_sko56_04_n


# Instancia del problema

## Importación de las librerias utilizadas

In [31]:
import random
import numpy as np

## Lectura de la instancia

In [32]:
def get_int_list(line):
  '''
  Recibe una cadena que contiene numeros enteros separados por ","
  retorna una lista con los elementos transformados a enteros.
  '''
  item_list = line.split(',')
  int_list = list(map(lambda item: int(item), item_list))
  return int_list

In [33]:
def read_instance(filename):
  with open(filename, 'r') as file:
    # Se lee la cantidad de tiendas
    n = int(file.readline())

    # Se leen los largos de cada tienda
    lengths_shops = get_int_list(file.readline())

    # Se leen la matriz de los clientes que visitan cada tienda
    matrix_clients = []
    for i in range(n):
      clients = get_int_list(file.readline())
      matrix_clients.append(clients)

  return n, lengths_shops, matrix_clients

In [34]:
n, lengths_shops, matrix_clients = read_instance(instance_file_name)

# Constante usada en el programa para crear y manipular los individuos
n_genes = n

## Funciones auxiliares

In [35]:
def create_individual(n_genes):
  return random.sample(range(n_genes), n_genes)

In [36]:
def create_population(n_genes, cant_individuals):
  return np.array([create_individual(n_genes) for i in range(cant_individuals)])

In [37]:
def calc_fitness_population(population):
  fitness = [calc_fitness(individual) for individual in population]
  return fitness

In [38]:
def get_best(population, fitness):
  best = population[0]
  F_min = fitness[0]
  length = len(population)

  for index in range(1, length):
    
    if fitness[index] < F_min:
      best = population[index]
      F_min = fitness[index]

  return best, F_min

## Funcion Objetivo

In [39]:
def get_distance(shop_i, shop_j):
  global lengths_shops
  half_start_shop = lengths_shops[shop_i] / 2
  half_final_shop = lengths_shops[shop_j] / 2

  distance_between_shops = sum(lengths_shops[shop_i + 1: shop_j])

  distance = half_start_shop + distance_between_shops + half_final_shop
  return distance

In [40]:
def calc_fitness(solution):
  global matrix_clients

  total_effort = 0
  for i in range(n - 1):
    for j in range(i + 1, n):
      x_i, x_j = solution[i], solution[j]
      current_effort = get_distance(x_i, x_j) * matrix_clients[x_i][x_j]
      total_effort += current_effort

  return total_effort

# Algoritmo Genético

## Definición de las operaciones geneticas


### Operador de selección

In [41]:
def get_min_index(fitness, index_participants):
  index_min = index_participants[0]
  min = fitness[index_participants[0]]

  for index in index_participants:
    if fitness[index] < min:
      index_min = index
      min = fitness[index]

  return index_min

In [42]:
def select(population, fitness, k, n_individuals):
  winners = []

  for i in range(n_individuals):
    index_participants = random.sample(range(n_individuals), k)
    
    winner_index = get_min_index(fitness, index_participants)
    winners.append(population[winner_index])

  return winners

### Operador de cruce

In [43]:
def copy_genes(individual_origin, individual_destiny, initial, quantity):
  index = initial
  while quantity > 0:
    individual_destiny[index] = individual_origin[index]

    index = (index + 1) % n_genes
    quantity -= 1

  return individual_destiny

In [44]:
def make_childrens(father_1, father_2):
  # Se los hijos sin genes todavia
  children_1 = [-1] * n_genes
  children_2 = [-1] * n_genes

  # Cantidad de genes que van a ser heredados a los hijos
  cant_genes_inherited = 4

  # Se selecciona un indice inicial al azar, del cual se empezara a copiar los genes
  initial_index = random.randint(0, n_genes - 1)

  # Se copian los genes heredados
  children_1 = copy_genes(father_1, children_1, initial_index, cant_genes_inherited)
  children_2 = copy_genes(father_2, children_2, initial_index, cant_genes_inherited)

  # Se inicializan los indices
  index = (initial_index + cant_genes_inherited) % n_genes
  index_children_1 = index
  index_children_2 = index

  # Mientras no se de la vuelta completa a los padres, se van copiando genes
  while True:

    # Si el gen actual del padre no esta en el hijo, se copia
    if father_1[index] not in children_2:
      children_2[index_children_2] = father_1[index]
      index_children_2 = (index_children_2 + 1) % n_genes

    if father_2[index] not in children_1:
      children_1[index_children_1] = father_2[index]
      index_children_1 = (index_children_1 + 1) % n_genes

    index = (index + 1) % n_genes

    if index == ((initial_index + cant_genes_inherited) % n_genes):
      break

  return children_1, children_2

In [45]:
def crossing(population, n_individuals, p_cross):
  for i in range(1, n_individuals, 2):
    rand = random.random()

    if rand < p_cross:
      population[i - 1], population[i] = make_childrens(population[i - 1], population[i])

  return population

### Operador de mutación

In [46]:
def make_mutation(individual):
  indexes = random.sample(range(n_genes), 2)
  
  individual[indexes[0]], individual[indexes[1]] = individual[indexes[1]], individual[indexes[0]]

  return individual

In [47]:
def mutate(population, n_individuals, p_mutate):
  for i in range(n_individuals):
    rand = random.random()

    if rand < p_mutate:
      population[i] = make_mutation(population[i])

  return population

## Definición del algoritmo


In [48]:
def run_genetic_algorithm(n_individuals, n_generations, p_cross, p_mutate):
  population = create_population(n, n_individuals)
  fitness = calc_fitness_population(population)

  best, min_fitness = get_best(population, fitness)

  for i in range(n_generations):
    population = select(population, fitness, 3, n_individuals)
    
    population = crossing(population, n_individuals, p_cross)
    
    population = mutate(population, n_individuals, p_mutate)

    fitness = calc_fitness_population(population)

    current_best, current_min_fitness = get_best(population, fitness)

    if current_min_fitness < min_fitness:
      best = current_best
      min_fitness = current_min_fitness

  return best, min_fitness

# Benchmark

In [63]:
POPULATION = 30
N_GENERATIONS = 100
PROB_CROSS = 0.7
PROB_MUT = 0.3

In [65]:
# Benchmark de una sola ejecucion
solution, fitness = run_genetic_algorithm(POPULATION, N_GENERATIONS, PROB_CROSS, PROB_MUT)
fitness

39065.0

In [70]:
# Benchmark para varias ejecuciones
QUANT_EJECUTIONS = 10
fitness = []

for i in range(QUANT_EJECUTIONS):
  current_fitness = run_genetic_algorithm(POPULATION, N_GENERATIONS, PROB_CROSS, PROB_MUT)[1]
  fitness.append(current_fitness)

mean = np.mean(fitness)
std_var = np.std(fitness)

print(f'Promedio: { mean } - Desviacion estandar: { std_var }')

Promedio: 42561.3 - Desviacion estandar: 3640.9525415748008
